In [ ]:
import os
import cv2
from os import path
import numpy as np
import splitfolders
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Dropout

#Divided Bottom_resized_png images file into 2 separate files as train and test datasets 
train_images = os.listdir('train_images/images')
test_images = os.listdir('test_images/images')
file_path = train_images + test_images
print(len(file_path))

In [ ]:
height = 224
width = 224
channels = 3
input_shape = (height,width,3)
epochs = 17
batch_size = 64
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(directory='train_images', 
                                                                         class_mode='categorical', 
                                                                         batch_size=batch_size,
                                                                         target_size=(height, width),
                                                                         color_mode="rgb",
                                                                         shuffle=True)
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(directory='test_images', 
                                                                         class_mode='categorical', 
                                                                         batch_size=batch_size,
                                                                         target_size=(height, width),
                                                                         color_mode="rgb",
                                                                         shuffle=True)

In [ ]:
input_shape = (224, 224, 3)
# Get VGG-16 Model
def getVGG16Model(lastFourTrainable=False):
    vgg_model = VGG16(weights='imagenet', input_shape=input_shape, include_top=True)

  # Make all layers untrainable
    for layer in vgg_model.layers[:]:
        layer.trainable = False

  # Add fully connected layer which have 1024 neuron to VGG-16 model
        output = vgg_model.get_layer('fc2').output
        output = Flatten(name='new_flatten')(output)
        output = Dense(units=1024, activation='relu', name='new_fc')(output)
        output = Dense(units=10, activation='softmax')(output)
        vgg_model = Model(vgg_model.input, output)

  # Make last 4 layers trainable if lastFourTrainable == True
    if lastFourTrainable == True:
        vgg_model.get_layer('block5_conv3').trainable = True
        vgg_model.get_layer('fc1').trainable = True
        vgg_model.get_layer('fc2').trainable = True
        vgg_model.get_layer('new_fc').trainable = True

  # Compile VGG-16 model
    vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    vgg_model.summary()

    return vgg_model

In [ ]:

# Get feature vector of an image by given model and img_path
def getFeatureVector(model, img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    feature_vector = model.predict(img.reshape(1, 224, 224, 3))
    return feature_vector


# Get cosine similarity between feature vectors A and B using cosine similarity
def getCosineSimilarity(A, B):
    cos_similarity = np.dot(A,B.T) / (np.linalg.norm(A)*np.linalg.norm(B)) # Get cosine similarity
    return cos_similarity[0][0]


# Function for get dataframe which contains the output features of given model
def getFeatureDataFrame(model):
    df = pd.DataFrame(columns=['file', 'features'])
    files = train_generator.filepaths

    df['file'] = files
    df['features'] = df.apply(lambda row: getFeatureVector(model, row['file']), axis=1) 

    print("All files added.")
    return df


# Get and plot 5 similar images for given image path and features dataframe
def getSimilarImages(img_file, features_df, model, model_name):
    img_features = getFeatureVector(model, img_file)
    features_df['similarity'] = features_df.apply(lambda row: getCosineSimilarity(img_features, np.asarray(row['features'])), axis=1)  
    sorted_df = features_df.sort_values(by='similarity', ascending=False)  
    plotSimilarImages(img_file, sorted_df.head(10), model_name)

In [ ]:
vgg_model_a = getVGG16Model(lastFourTrainable=False)
feature_model_vgg_a = Model(inputs=vgg_model_a.input, outputs=vgg_model_a.get_layer('new_fc').output)

df = getFeatureDataFrame(feature_model_vgg_a)

In [ ]:
# Plot similar 10 images with given image and similar images dataframe
def plotSimilarImages(img_file, similar_df, model_name):
    img = cv2.imread(img_file)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    test_fig, test_axarr =  plt.subplots()
    test_axarr.imshow(img)
    test_axarr.set_title("TEST IMAGE - " + model_name)
    test_axarr.axis('off')

    j, k, m = 0, 0, 0
    fig, axarr = plt.subplots(2,5, figsize=(15,15))
    for index, sim in similar_df.iterrows():
        filepath = sim['file']
        similarity = sim['similarity']

        similar = cv2.imread(filepath)
        similar = cv2.resize(similar, (224, 224))
        similar = cv2.cvtColor(similar, cv2.COLOR_BGR2RGB)
        axarr[k,m].imshow(similar)
        axarr[k,m].set_title("Similarity: %.3f" % similarity)
        axarr[k,m].axis('off')

        m += 1
        if m == 5 and k !=1:
            k += 1
            m = 0

        j += 1
        if j == 10:
            break

    plt.tight_layout()
    plt.show()

In [ ]:
# Test images path
feature_test_files = test_generator.filepaths


# Get 10 similar images of test images for VGG-16 (a)
for file in feature_test_files:
    getSimilarImages(file, df, feature_model_vgg_a, 'VGG-16 (a)')